# 基本概念

自然语言处理（NLP）推荐系统结合了NLP的技术和推荐系统的需求，目的是根据用户的需求和行为，提供个性化的内容推荐。NLP推荐系统通常会使用文本数据、用户偏好、历史行为等信息，来为用户推荐相关的内容、商品或服务。

常见的NLP推荐系统技术包括：

1. 基于内容的推荐：
    - 特征提取：首先，从文本数据中提取特征（如关键词、主题、情感等）。这通常通过词嵌入（Word2Vec、GloVe等）或BERT等Transformer模型来实现。
    - 文本相似度计算：通过计算文本之间的相似度，推荐与用户过往兴趣相关的内容。例如，基于TF-IDF、余弦相似度等方法，找出最相似的内容。
2. 协同过滤（Collaborative Filtering）：
    - 基于用户的协同过滤：根据与用户兴趣相似的其他用户的行为，推荐内容。例如，基于历史行为（点击、评分等）找到相似的用户，推荐他们喜欢的内容。
    - 基于项目的协同过滤：通过分析项目（如商品、文章等）之间的相似性，进行推荐。对于用户未曾接触的项目，推荐与其历史偏好相似的项目。
3. 混合推荐系统：
    - 结合多种推荐技术（如基于内容和协同过滤），通过加权融合或模型组合，提高推荐的准确度和多样性。
4. 基于深度学习的推荐：
    - 神经网络模型：如使用RNN、LSTM、Transformer等网络架构，对用户的历史行为、文本内容进行建模。
    - BERT模型：通过预训练的BERT模型捕捉语境信息，理解用户的潜在需求。
5. 情感分析与情绪驱动的推荐：
    - 通过分析用户对内容的情感反应（如评论或反馈中的情感倾向），为用户推荐符合其当前情绪需求的内容。例如，情感分析可以帮助识别用户喜欢的电影类型或音乐风格。
6. 序列化推荐：
    - 序列建模：通过捕捉用户的行为序列（如浏览历史、购买记录），使用LSTM、Transformer等模型预测下一步可能感兴趣的内容。
    - 个性化时序推荐：结合时间戳、上下文信息来进行推荐，使得推荐系统更符合用户的实时需求。

# 关键词提取

# 文本相似度

# 协同过滤

协同过滤（Collaborative Filtering, CF）是推荐系统中常用的一种技术，它通过分析用户之间或物品之间的相似性来进行推荐，主要分为基于用户的协同过滤和基于物品的协同过滤。协同过滤的基本思想是“与我相似的人喜欢的东西，我也可能喜欢”，或“与我相似的商品，可能是我感兴趣的商品”。

## 基于用户的协同过滤

基于用户的协同过滤通过找出兴趣相似的用户来推荐内容。例如，如果用户A和用户B在过去的行为上表现出相似的兴趣，那么A喜欢的内容也可能会推荐给B。

工作原理：

- 计算相似度：首先计算用户之间的相似度，常用的方法是计算两位用户行为的相似度，如使用余弦相似度、皮尔逊相关系数等。
- 相似用户推荐：通过找到与目标用户兴趣相似的其他用户，推荐这些相似用户喜欢的物品。
  
步骤：

- 构建用户-物品矩阵，其中矩阵中的每个元素表示用户对物品的评分（或行为，如点击、购买等）。
- 计算用户之间的相似度。
- 找到与目标用户最相似的用户，生成推荐列表。
  
优点：

- 简单直观。
- 不需要对物品本身进行深入分析，适用于内容未知的推荐。
  
缺点：

- 稀疏性问题：大多数用户和物品之间的评分数据较少，导致相似度计算困难。
- 扩展性差：当用户数和物品数较大时，计算和存储相似度矩阵非常昂贵。
- 冷启动问题：对于新用户或新物品没有足够的历史数据，导致推荐效果较差。

In [1]:
import numpy as np
import pandas as pd

users = ['User1', 'User2', 'User3', 'User4', 'User5']
items = ['ItemA', 'ItemB', 'ItemC', 'ItemD', 'ItemE']

# 1.构造用户购买记录数据集
datasets = [
    [1,0,1,1,0],
    [1,0,0,1,1],
    [1,0,1,0,0],
    [0,1,0,1,1],
    [1,1,1,0,1],
]
df = pd.DataFrame(datasets, columns=items, index=users)
print(df)

# 2.计算用户间相似度
user_similar = []
for user in df.index:
    # 计算每个用户和其他用户间的相似度
    similarity_list = []
    for other_user in df.index:
        user_list = list(df.loc[user])
        other_user_list = list(df.loc[other_user])
        common_num = sum([ 1 for i in range(len(user_list)) if user_list[i] + other_user_list[i] == 2 ])
        similarity = (common_num / sum(user_list)) * (common_num / sum(other_user_list))
        similarity_list.append(similarity)
    # 构建用户间相似度矩阵
    user_similar.append(similarity_list)

user_similar = pd.DataFrame(user_similar, columns=users, index=users)

print("用户之间的两两相似度：")
print(user_similar)

# 3.构造每个用户相似度topN的用户字典
topN_users = {}
for i in user_similar.index:
    # 取出每一列数据，并删除自身，然后排序数据
    _df = user_similar.loc[i].drop([i])
    _df_sorted = _df.sort_values(ascending=False)

    top2 = list(_df_sorted.index[:2])
    topN_users[i] = top2

print("Top2相似用户：")
print(topN_users)

# 4.构建推荐结果
rs_results = {}
for user, sim_users in topN_users.items():
    # 存储每个用户推荐结果
    rs_result = set()   
    # 找到最相似的topN用户购买过的物品
    for sim_user in sim_users:
        rs_result = rs_result.union(set(df.loc[sim_user].replace(0, np.nan).dropna().index))
    # 过滤掉已经购买过的物品
    rs_result -= set(df.loc[user].replace(0, np.nan).dropna().index)

    rs_results[user] = rs_result

print("最终推荐结果：")
print(rs_results)

       ItemA  ItemB  ItemC  ItemD  ItemE
User1      1      0      1      1      0
User2      1      0      0      1      1
User3      1      0      1      0      0
User4      0      1      0      1      1
User5      1      1      1      0      1
用户之间的两两相似度：
          User1     User2     User3     User4     User5
User1  1.000000  0.444444  0.666667  0.111111  0.333333
User2  0.444444  1.000000  0.166667  0.444444  0.333333
User3  0.666667  0.166667  1.000000  0.000000  0.500000
User4  0.111111  0.444444  0.000000  1.000000  0.333333
User5  0.333333  0.333333  0.500000  0.333333  1.000000
Top2相似用户：
{'User1': ['User3', 'User2'], 'User2': ['User1', 'User4'], 'User3': ['User1', 'User5'], 'User4': ['User2', 'User5'], 'User5': ['User3', 'User1']}
最终推荐结果：
{'User1': {'ItemE'}, 'User2': {'ItemB', 'ItemC'}, 'User3': {'ItemD', 'ItemB', 'ItemE'}, 'User4': {'ItemA', 'ItemC'}, 'User5': {'ItemD'}}


## 基于物品的协同过滤

基于物品的协同过滤的思路是通过分析物品之间的相似性来为用户推荐物品。如果一个用户喜欢某个物品，那么推荐与该物品相似的其他物品。相比于基于用户的协同过滤，基于物品的协同过滤在计算时通常更稳定，且能有效解决稀疏性问题。

工作原理：

- 计算相似度：通过分析物品之间的共现关系，计算物品之间的相似度。例如，用户A和用户B都喜欢物品X和物品Y，那么X和Y就是相似的。
- 基于物品推荐：通过找出目标物品与其他物品的相似性，推荐与目标物品相似的其他物品。
  
步骤：

- 构建用户-物品评分矩阵，计算物品之间的相似度。
- 对用户历史评分的物品，找到与这些物品相似的其他物品。
- 根据相似度推荐物品。
  
优点：

- 更稳定：物品的相似度变化较少，不受单一用户行为的影响。
- 推荐质量较高：基于物品的协同过滤能有效解决稀疏性问题。
  
缺点：

- 需要计算所有物品之间的相似度，计算量大。
- 冷启动问题：对于新物品没有足够的历史数据，难以计算相似度。

In [2]:
#Item-Based CF 基于物品的协同过滤推荐
import numpy as np
import pandas as pd

users = ['User1', 'User2', 'User3', 'User4', 'User5']
items = ['ItemA', 'ItemB', 'ItemC', 'ItemD', 'ItemE']

# 1.构造用户购买记录数据集
datasets = [
    [1,0,1,1,0],
    [1,0,0,1,1],
    [1,0,1,0,0],
    [0,1,0,1,1],
    [1,1,1,0,1],
]
df = pd.DataFrame(datasets, columns=items, index=users)
print(df)

# 2.计算物品间相似度
item_similar = []
for item in df.T.index:
    # 计算每个用户和其他用户间的相似度
    similarity_list = []
    for other_item in df.T.index:
        item_list = list(df.T.loc[item])
        other_item_list = list(df.T.loc[other_item])
        common_num = sum([ 1 for i in range(len(item_list)) if item_list[i] + other_item_list[i] == 2 ])
        similarity = (common_num / sum(item_list)) * (common_num / sum(other_item_list))
        similarity_list.append(similarity)
    # 构建用户间相似度矩阵
    item_similar.append(similarity_list)

item_similar = pd.DataFrame(item_similar, columns=items, index=items)

print("物品之间的两两相似度：")
print(item_similar)

# 3.构造每个物品相似度topN的物品字典
topN_items = {}
for i in item_similar.index:
    # 取出每一列数据，并删除自身，然后排序数据
    _df = item_similar.loc[i].drop([i])
    _df_sorted = _df.sort_values(ascending=False)

    top2 = list(_df_sorted.index[:2])
    topN_items[i] = top2

print("Top2相似物品：")
print(topN_items)

# 4.构建推荐结果
rs_results = {}
for user in df.index:
    # 存储每个用户推荐结果
    rs_result = set() 
    # 根据每个物品找出最相似的topN物品
    for item in df.loc[user].replace(0, np.nan).dropna().index: 
        rs_result = rs_result.union(topN_items[item])
    # 过滤掉用户已经购买过的物品
    rs_result -= set(df.loc[user].replace(0, np.nan).dropna().index)
    
    rs_results[user] = rs_result

print("最终推荐结果：")
print(rs_results)

       ItemA  ItemB  ItemC  ItemD  ItemE
User1      1      0      1      1      0
User2      1      0      0      1      1
User3      1      0      1      0      0
User4      0      1      0      1      1
User5      1      1      1      0      1
物品之间的两两相似度：
          ItemA     ItemB     ItemC     ItemD     ItemE
ItemA  1.000000  0.125000  0.750000  0.333333  0.333333
ItemB  0.125000  1.000000  0.166667  0.166667  0.666667
ItemC  0.750000  0.166667  1.000000  0.111111  0.111111
ItemD  0.333333  0.166667  0.111111  1.000000  0.444444
ItemE  0.333333  0.666667  0.111111  0.444444  1.000000
Top2相似物品：
{'ItemA': ['ItemC', 'ItemD'], 'ItemB': ['ItemE', 'ItemC'], 'ItemC': ['ItemA', 'ItemB'], 'ItemD': ['ItemE', 'ItemA'], 'ItemE': ['ItemB', 'ItemD']}
最终推荐结果：
{'User1': {'ItemB', 'ItemE'}, 'User2': {'ItemC', 'ItemB'}, 'User3': {'ItemB', 'ItemD'}, 'User4': {'ItemA', 'ItemC'}, 'User5': {'ItemD'}}
